# 1. 목표
> Telco Customer Churn 데이터 활용으로:

- 이탈 고객과 비이탈 고객의 특성을 비교   
- **이탈 위험 고객군을 식별**   
- **고객 유치 전략(예: 할인, 혜택 제공, 맞춤 마케팅)**의 기반 데이터로 활용

# 2. EDA (탐색적 데이터 분석)
- 결측치, 이상치 확인 및 처리    
- 이탈 여부(`Churn`) 비율 확인    
- 연속형 변수(예: `MonthlyCharges`, `TotalCharges`, `tenure`)에 따른 이탈 여부 분포 비교    
- 범주형 변수(예: `Contract`, `InternetService`, `PaymentMethod`)별 이탈률 시각화

# 3. 시각화
- `sns.countplot` / `sns.barplot` / `plt.hist` / `plt.boxplot`으로 이탈 여부별 변수 비교   
- Heatmap으로 변수 간 상관관계 확인

# 4. 고객군 식별 로직 설계
- 특정 조건(예: 단기 가입자, 월 요금 높은데 계약기간 짧음, 전자결제 사용, 월별 결제 등)으로 조건 필터링하여 이탈 위험 고객군 추출

# 5. 추가적으로 기술을 익히면 좋을 것
 - `RandomForest`, `LogisticRegression`, `XGBoost` 등을 사용해 이탈 예측 모델까지 확장 가능

In [1]:
# 라이브러리 불러오기

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
# 경고 안 뜨게 하기

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 데이터 파일 불러오기

df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.shape

(7043, 21)

In [4]:

# 시스템 환경에 따라 폰트를 자동으로 설정해주는 함수
def get_font_family():
    import platform
    system_name = platform.system()

    if system_name == "Darwin":
        return "AppleGothic"
    elif system_name == "Windows":
        return "Malgun Gothic"
    else:
        # Linux (Colab)
        !apt-get install fonts-nanum -qq > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont

        return "NanumBarunGothic"

# 한글 폰트 적용
plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

# 레티나 디스플레이 설정 (선명하게)
%config InlineBackend.figure_format = 'retina'



In [5]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


# 1️⃣ 결측치 및 데이터 타입 확인 후 정제
- `isnull().sum()`, `info()`, `describe()` 확인   
- `TotalCharges` 문자열 → 숫자 변환   
- 결측값/공백 처리 및 타입 정리

In [9]:


# 데이터 타입 및 결측치 확인
df.info()

# 결측치 확인 (공백 포함)
missing_check = df.isnull().sum()

# TotalCharges의 공백을 NaN 처리 후 수치형(float)으로 변환(우선 공백 확인)

blank_totalcharges = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isna()]

# TotalCharges를 수치형으로 변환, errors='coerce'로 공백을 NaN으로 처리
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 변환 후 결측치 재확인
missing_after_conversion = df.isnull().sum()

# 결측치 행 제거 후 데이터 클린 완료
df_clean = df.dropna()

# 결과 요약 확인
df_clean.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 
